### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
from bs4 import BeautifulSoup
from collections import Counter

In [2]:
Match = pd.read_csv('data/match_with_stats.csv')

In [3]:
Match.columns.values

array(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'home_player_Y1',
       'home_player_Y2', 'home_player_Y3', 'home_player_Y4',
       'home_player_Y5', 'home_player_Y6', 'home_player_Y7',
       'home_player_Y8', 'home_player_Y9', 'home_player_Y10',
       'home_player_Y11', 'away_player_Y1', 'away_player_Y2',
       'away_player_Y3', 'away_player_Y4', 'away_player_Y5',
       'away_player_Y6', 'away_player_Y7', 'away_player_Y8',
       'away_player_Y9', 'away_player_Y10', 'away_player_Y11',
       'home_player_1', 'home_player_2', 'home_player_3', 'home_player_4',
       'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8',
       'home_player_9', 'home_player_10', 'home_player_11',
       'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
       'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',
       'away_playe

In [4]:
####Most of these stats are not consistent throughout games and are freak events
###Should not be used to predict match outcome
###Relevant Columns
rel_stat_cols = ['possession', 'foulcommit']

In [5]:
from xml.etree import ElementTree as ET

In [6]:
Markup_Stats = pd.DataFrame(columns = ['home_possession', 'away_possession', 'home_fouls', 'away_fouls'])

In [7]:
for i in range(len(Match)):
    root = ET.fromstring(Match['possession'][i])
    for el in root.findall('value'):
        full_t = int(el.find('elapsed').text)
        if full_t == 90:
            h_pos = el.find('homepos')
            a_pos = el.find('awaypos')
            if h_pos is not None:
                home_poss = int(h_pos.text)
            else:
                h_pos = None
            if a_pos is not None:
                away_poss = int(a_pos.text)
            else:
                a_pos = None
    #print({'home_possession': home_poss, 'away_possession': away_poss})
    #Markup_Stats.append({'home_possession': home_poss, 'away_possession': away_poss}, ignore_index=True)
    root2 = ET.fromstring(Match['foulcommit'][i])
    foul_list = []
    for el in root2.findall('value'):
        foul_comm = el.find('type').text
        if foul_comm == 'foulcommit':
            node = el.find('team')
            if node is not None:
                foul_inst = int(node.text)
                foul_list.append(foul_inst)
            else:
                node = None
    foul_count = list(dict(Counter(foul_list)).items())
    if not foul_count:
        #print({'home_fouls': 0, 'away_fouls': 0})
        Markup_Stats = Markup_Stats.append({'home_possession': home_poss, 'away_possession': away_poss, 'home_fouls': 0, 'away_fouls': 0}, ignore_index=True)
    elif len(foul_count) == 1:
        #print({'home_fouls': foul_count[0][1], 'away_fouls': 0})
        Markup_Stats = Markup_Stats.append({'home_possession': home_poss, 'away_possession': away_poss, 'home_fouls': foul_count[0][1], 'away_fouls': 0}, ignore_index=True)
    elif foul_count:
        #print({'home_fouls': foul_count[0][1], 'away_fouls': foul_count[1][1]})
        Markup_Stats = Markup_Stats.append({'home_possession': home_poss, 'away_possession': away_poss, 'home_fouls': foul_count[0][1], 'away_fouls': foul_count[1][1]}, ignore_index=True)
    #print(i)
#2962 problematic

In [8]:
root3 = ET.fromstring(Match['foulcommit'][5019])
foul_list = []
for el in root3.findall('value'):
    foul_comm = el.find('type').text
    if foul_comm == 'foulcommit':
        node = el.find('team')
        if node is not None:
            foul_inst = int(node.text)
            foul_list.append(foul_inst)
        else:
            node = None
foul_count = list(dict(Counter(foul_list)).items())
print(foul_count)
if not foul_count:
    print({'home_fouls': 0, 'away_fouls': 0})
elif len(foul_count) == 1:
    print({'home_fouls': foul_count[0][1], 'away_fouls': 0})
elif foul_count:
    print({'home_fouls': foul_count[0][1], 'away_fouls': foul_count[1][1]})

[(10189, 3)]
{'home_fouls': 3, 'away_fouls': 0}


In [9]:
Markup_Stats.shape

(13321, 4)

In [10]:
Markup_Stats.head(5)

,home_possession,away_possession,home_fouls,away_fouls
0,55,45,11,16
1,66,34,9,11
2,46,54,12,13
3,52,48,14,13
4,51,49,11,11


In [11]:
Markup_Stats.tail(100)

,home_possession,away_possession,home_fouls,away_fouls
13221,52,48,11,8
13222,62,38,12,14
13223,62,38,0,0
13224,62,38,0,0
13225,62,38,0,0
...,...,...,...,...
13316,62,38,0,0
13317,62,38,0,0
13318,62,38,0,0
13319,62,38,0,0


In [12]:
Markup_Stats.iloc[13223:].value_counts()

home_possession  away_possession  home_fouls  away_fouls
62               38               0           0             98
dtype: int64

Every match from 13223 onwards seems to be identical/repetitive data

In [13]:
red_mark_stats = Markup_Stats.iloc[:13223]

In [14]:
red_mark_stats.shape

(13223, 4)

In [15]:
red_mark_stats.tail(5)

,home_possession,away_possession,home_fouls,away_fouls
13218,49,51,26,10
13219,50,50,11,10
13220,52,48,8,15
13221,52,48,11,8
13222,62,38,12,14


In [16]:
red_mark_stats.head(5)

,home_possession,away_possession,home_fouls,away_fouls
0,55,45,11,16
1,66,34,9,11
2,46,54,12,13
3,52,48,14,13
4,51,49,11,11


In [17]:
 red_mark_stats.to_csv('data/' + 'reduced_match_stats.csv')